In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
np.random.seed(123)

In [ ]:
iters = 50000
N = 100
t = np.arange(0,iters+1,1)

n_e = ((N-1)/(N))**t
exp_e = np.exp(-t/N) 

replications = 10000

h_mat = np.zeros((replications, iters+1))
e1 = np.zeros_like(h_mat)
e2 = np.zeros_like(h_mat) 

x_log = np.ones_like(h_mat)

for r in range(replications):
    #if r%100 == 0: print(r)     
    hist = [1] 
    x = np.random.rand(N)
    
    for k in range(iters):
        x = np.sort(x)
        v = x[-1] # maximum x is minimum L
        x_log[r,k+1] = v 
        hist.append(v)
        x[-1] = v * np.random.rand() # draw from U(L,1)
    
    h = np.array(hist) 
    h_mat[r, :] = h
    
print("Done!") 

In [ ]:
def s2l(num):
    # Split the scientific notation into base and exponent
    base, exponent = f"{num:.2e}".split('e')
    
    # Format base and exponent in LaTeX format
    base = float(base)  # Convert base to float for proper formatting
    return f"{base:.1f}" + r" \times " + f"10^{{{int(exponent)}}}"

vals = [1000 + 2000*k for k in range(25)]
print('iter, exp, walter,  mean', 'median') 
for v in vals: 
    hh = h_mat[:, v+1] 
    print(f"{v} & ${s2l(exp_e[v])}$ & ${s2l(n_e[v])}$ & ${s2l(np.mean(hh))}$ & ${s2l(np.median(hh))}$" + r" \\")

In [ ]:
L = lambda x, v: 0.1*(1-x) + 1.9*(x<v)*(v-x)/(v**2)

def L2(x, v):
    val = 0.1*(1-x) + 1.9*np.exp(np.log((x<v)*np.abs(v-x)) - 2*np.log(v))
    return val 

In [ ]:
def integrate(x, v, N, exp_mode=False): 
    res = np.zeros(x.shape[0]) 
    Lvals = L2(x, v) 
    
    if exp_mode:
        w = lambda t: np.exp(-t/N) 
    else: 
        w = lambda t: ((N-1)/N)**t
  
    for t in range(1,x.shape[1]):
        sh = (w(t-1)-w(t))
        res += Lvals[:,t] * sh

    return res 

In [ ]:
p = [10**(-k) for k in range(1,150,2)]

Z_e = np.zeros((len(p), replications))
Z_n = np.zeros_like(Z_e)

for k, _p in enumerate(p): 
    Z_e[k,:] = integrate(x_log, N=100, v = _p, exp_mode=True)
    Z_n[k,:] = integrate(x_log, N=100, v = _p, exp_mode=False)            

# note that divide by zero warning is just for when the second term in L2 is zero (np.exp(-np.inf))

In [ ]:
np.save('Z_e2.npy', Z_e)
np.save('Z_n2.npy', Z_n)

In [ ]:
#Z_e = np.load('Z_e.npy') 
#Z_n = np.load('Z_n.npy')

In [ ]:
meanZ_e = np.mean(Z_e, axis=1)
stdZ_e = np.std(Z_e, axis=1)
meanZ_n = np.mean(Z_n, axis=1) 
stdZ_n = np.std(Z_n, axis=1) 

mean_log_Z_e = np.mean(np.log(Z_e), axis=1)
std_log_Z_e = np.std(np.log(Z_e), axis=1)
mean_log_Z_n = np.mean(np.log(Z_n), axis=1) 
std_log_Z_n = np.std(np.log(Z_n), axis=1) 

In [ ]:
# # logscale computation, gives same results for this example but much slower!

# from scipy.special import logsumexp
# def integrate_logscale(x, v, N, exp_mode=False): 
    
#     res = -np.inf * np.ones(x.shape[0]) 
#     Lvals = L2(x, v) 
    
#     if exp_mode:
#         w = lambda t: np.exp(-t/N) 
#     else: 
#         w = lambda t: ((N-1)/N)**t
  
#     for t in range(1,x.shape[1]):
#         sh = (w(t-1)-w(t))
#         # update z = z + w*L is now log z = log ( exp(log(z)) + exp(log(w*L)) ) 
#         res = logsumexp([res, np.log(Lvals[:,t])+np.log(sh)], axis=0) 

#     return res    

# p = [10**(-k) for k in range(1,150,5)]

# Z_e = np.zeros((len(p), replications))
# Z_n = np.zeros_like(Z_e)

# for k, _p in enumerate(p): 
#     print(_p)
#     Z_e[k,:] = integrate_logscale(x_log, N=100, v = _p, exp_mode=True)
#     Z_n[k,:] = integrate_logscale(x_log, N=100, v = _p, exp_mode=False)            

# meanZ_e = np.mean(np.exp(Z_e), axis=1)
# stdZ_e = np.std(np.exp(Z_e), axis=1)
# meanZ_n = np.mean(np.exp(Z_n), axis=1) 
# stdZ_n = np.std(np.exp(Z_n), axis=1) 

# mean_log_Z_e = np.mean(Z_e, axis=1)
# std_log_Z_e = np.std(Z_e, axis=1)
# mean_log_Z_n = np.mean(Z_n, axis=1) 
# std_log_Z_n = np.std(Z_n, axis=1) 

# print(meanZ_e, std_log_Z_e)

In [ ]:
#plt.close() 
p = [10**(-k) for k in range(1,150,2)]
plt.figure(figsize=(10,5))
plt.subplot(1,2,1)

plt.axhline(y = 1, color = 'k', linestyle = ':')
plt.semilogx(p, meanZ_n, 'b-', linewidth=3, label = r'$((N-1)/N)^t$')
plt.semilogx(p, meanZ_n + stdZ_n, 'b-', alpha=0.5)
plt.semilogx(p, meanZ_n - stdZ_n, 'b-', alpha=0.5)

plt.semilogx(p, meanZ_e, 'r--', linewidth=3, label=r'$\exp(-t/N)$')
plt.semilogx(p, meanZ_e + stdZ_e, 'r--', alpha=0.5)
plt.semilogx(p, meanZ_e - stdZ_e, 'r--', alpha=0.5)


plt.xlabel(r'$v$')
plt.title(r'$\mathcal{Z}$')
plt.legend() 

plt.subplot(1,2,2)
plt.axhline(y = 0, color = 'k', linestyle = ':')
plt.semilogx(p, mean_log_Z_n, 'b-', linewidth=3, label=r'$((N-1)/N)^t$')
plt.semilogx(p, mean_log_Z_n + std_log_Z_n, 'b-', alpha=0.5)
plt.semilogx(p, mean_log_Z_n - std_log_Z_n, 'b-', alpha=0.5)

plt.semilogx(p, mean_log_Z_e, 'r--', linewidth=3, label = r'$\exp(-t/N)$')
plt.semilogx(p, mean_log_Z_e + std_log_Z_e, 'r--', alpha=0.5)
plt.semilogx(p, mean_log_Z_e - std_log_Z_e, 'r--', alpha=0.5)
plt.xlabel(r'$v$')
plt.title(r'$\log \mathcal{Z}$')
plt.legend() 
plt.show()

#plt.savefig('Zvar.pdf')

In [ ]:
medZ_e = np.median(Z_e, axis=1)
lowZ_e = np.quantile(Z_e, 0.05, axis=1)
lowZ_e2 = np.quantile(Z_e, 0.25, axis=1)
upZ_e = np.quantile(Z_e, 0.95, axis=1)
upZ_e2 = np.quantile(Z_e, 0.75, axis=1)

medZ_n = np.median(Z_n, axis=1)
lowZ_n = np.quantile(Z_n, 0.05, axis=1)
lowZ_n2 = np.quantile(Z_n, 0.25, axis=1)

upZ_n = np.quantile(Z_n, 0.95, axis=1)
upZ_n2 = np.quantile(Z_n, 0.75, axis=1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, sharex=True, sharey=True, figsize=(10,5))

plt.subplot(1,2,1)
plt.semilogx(p, medZ_e, 'r') 
plt.fill_between(p, lowZ_e, upZ_e,  color='r', alpha=0.1)
plt.fill_between(p, lowZ_e2, upZ_e2, color='r', alpha=0.15)
plt.axhline(y = 1, color = 'k', linestyle = ':')
plt.title(r'$\exp(-t/N)$')
plt.xlabel(r'$v$')

plt.subplot(1,2,2)
plt.semilogx(p, medZ_n, 'b')
plt.fill_between(p, lowZ_n, upZ_n,  color='b', alpha=0.1)
plt.fill_between(p, lowZ_n2, upZ_n2, color='b', alpha=0.15)
plt.axhline(y = 1, color = 'k', linestyle = ':')
plt.title(r'$((N-1)/N)^t$')
plt.xlabel(r'$v$')
plt.show()
#plt.savefig('Zquantiles.pdf') 